In [1]:
import torch
from torch_geometric.loader import DataLoader,RandomNodeSampler, NeighborLoader
from torch_geometric.data import Data
from models import *
from utils import *
from preprocessing import *
import networkx as nx

epochs = 30
T = 15
dataset="PA_5"
subset_size=0.3
infect_rate=0.01
k = 100
delay =5
tau = 7 


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_ranking = Ranking_Module().to(device)
optimizer = torch.optim.Adam(model_ranking.parameters(), lr=0.001, weight_decay=5e-4)

# model_diffuse = Local_Diffusion(data).to(device)
# model_long = Long_Information().to(device)
# model_hidden = Hidden_State().to(device)
# model_score = Score_Net().to(device)


# if dataset=="data/ca-GrQc.txt":
#     edges = get_Edges("data/ca-GrQc.txt")
#     with open('node_features.npy', 'rb') as f:
#         node_features = np.load(f) 
# elif dataset=="PA_5":
#     edges = get_Edges("",PA=True,PA_num = 5)
#     node_features = get_Node_features("",edes=edges)

# optimizer_diffuse = torch.optim.Adam(model_diffuse.parameters(), lr=0.01, weight_decay=5e-4)
# optimizer_long = torch.optim.Adam(model_long.parameters(), lr=0.01, weight_decay=5e-4)
# optimizer_hidden = torch.optim.Adam(model_long.parameters(), lr=0.01, weight_decay=5e-4)


train_running_loss = 0

for i in range(epochs):
    print("-------------------")
    print("Epoch",i)
    if dataset=="data/ca-GrQc.txt":
        edges = get_Edges("data/ca-GrQc.txt")
        with open('node_features.npy', 'rb') as f:
            node_features = np.load(f) 
    elif "PA" in dataset:
        edges = get_Edges("",PA=True,PA_num = int(dataset[-1]))
        node_features = get_Node_features("",edges=edges)
    

    hidden_states = torch.rand(len(node_features), 64)
    hidden_states= hidden_states.to(device)

    loss_fc = nn.BCELoss() #.to(device)

    node_features = torch.from_numpy(node_features).to(device).float()
    # All nodes are untested from the beginning
    node_features[:,4]  = node_features[:,4] + 1 
    
    # For each epoch, we randomly sample from the PA graph and construct new node statuses
    node_status = status_setup(len(node_features),infect_rate)
    temporal_edges = sample(T, edges, subset_size)
    t_edges_index = temporal_edges[:,:,0:2].astype(int)
    t_edges_attr =  temporal_edges[:,:,2]
    for t in range(T):
        print("-------------------")
        print("t = ",t)
        if t!=0:
            SEIR_update(node_status,t_edges_index[t-1], t_edges_attr[t-1],t,delay)
        #status_summary(node_status)
        #node_features = torch.from_numpy(node_features).to(device).float()
        edges_index = torch.from_numpy(t_edges_index[t]).to(device).long()
        edges_attr = torch.from_numpy(t_edges_attr[t]).to(device).float()
        data = Data(x = node_features, edge_index=edges_index.t().contiguous(),edge_attr= edges_attr)

        # data = Data(x = node_features, edge_index=t_edges_index[t].t().contiguous(),edge_attr= t_edges_attr[t])
        # data.x = data.x.to(device).to(dtype=torch.float32)
        # data.edge_index = data.edge_index.to(device).long()
        # data.edge_attr = data.edge_attr.to(device).to(dtype=torch.float32)
                
        multi_edges, multi_attr = gen_multi_G(t_edges_index,t_edges_attr,t,tau)
        multi_edges = torch.from_numpy(multi_edges).to(device)
        multi_attr  = torch.from_numpy(multi_attr).to(device)
        data_multi = Data(x = node_features, edge_index=multi_edges.t().contiguous(),edge_attr= multi_attr)
        
        scores, hidden_states = model_ranking(data,data_multi,hidden_states)
        
        labels = 1-node_status[:,0]   
        labels = torch.from_numpy(labels)
        labels = labels.reshape(-1,1)
        labels = labels.to(device).float()
        loss = loss_fc(scores, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        hidden_states = hidden_states.detach()
        # Pick top k that are not isolated 
        probability = k_node_sample_prob(scores)
        counter =k
        isolated_index = np.where(node_status[:,3] == 1)[0]

        # don't calculate loss on isolated nodes
        # scores_non_isolated= scores[np.where(node_status[:,3] == 0)[0]]
        # labels_non_isolated = labels[np.where(node_status[:,3] == 0)[0]]

        # don't intervene on isolated node
        probability[isolated_index]=0
        while counter>0:
            prev_test_positive = torch.where(node_features[:,5]==1)[0]
            node_features[prev_test_positive,5]=0
            node_features[prev_test_positive,7]=1
            index = np.argmax(np.random.multinomial(1,probability.cpu().detach().numpy().reshape(-1,),1))
            node_features[index,4]=0 
            if node_status[index,2]==1:
                node_status[index,2]=0
                node_status[index,-1]=1
                node_features[index,5]=1
                # detection_rate[t]+=1
            elif node_status[index,1]>0:
                node_status[index,1]=0
                node_status[index,-1]=1
                node_features[index,5]=1
                # detection_rate[t]+=1
            else:
                node_features[index,6]=1
            probability[index]=0
            counter-=1
        train_running_loss += loss.detach().item()
torch.save(model_ranking.state_dict(), 'model_ranking_30.pth')
model_ranking.eval()
if dataset=="data/ca-GrQc.txt":
        edges = get_Edges("data/ca-GrQc.txt")
        with open('node_features.npy', 'rb') as f:
            node_features = np.load(f) 
elif "PA" in dataset:
    edges = get_Edges("",PA=True,PA_num = int(dataset[-1]))
    node_features = get_Node_features("",edges=edges)


hidden_states = torch.rand(len(node_features), 64)
hidden_states= hidden_states.to(device)

loss_fc = nn.BCELoss() #.to(device)

node_features = torch.from_numpy(node_features).to(device).float()
# All nodes are untested from the beginning
node_features[:,4]  = node_features[:,4] + 1 

# For each epoch, we randomly sample from the PA graph and construct new node statuses
node_status = status_setup(len(node_features),infect_rate)
temporal_edges = sample(T, edges, subset_size)
t_edges_index = temporal_edges[:,:,0:2].astype(int)
t_edges_attr =  temporal_edges[:,:,2]
for t in range(T):
    print("--------Testing-----------")
    print("t = ",t)
    if t!=0:
        SEIR_update(node_status,t_edges_index[t-1], t_edges_attr[t-1],t,delay)
    status_summary(node_status)
    #node_features = torch.from_numpy(node_features).to(device).float()
    edges_index = torch.from_numpy(t_edges_index[t]).to(device).long()
    edges_attr = torch.from_numpy(t_edges_attr[t]).to(device).float()
    data = Data(x = node_features, edge_index=edges_index.t().contiguous(),edge_attr= edges_attr)
            
    multi_edges, multi_attr = gen_multi_G(t_edges_index,t_edges_attr,t,tau)
    multi_edges = torch.from_numpy(multi_edges).to(device)
    multi_attr  = torch.from_numpy(multi_attr).to(device)
    data_multi = Data(x = node_features, edge_index=multi_edges.t().contiguous(),edge_attr= multi_attr)
    
    scores, hidden_states = model_ranking(data,data_multi,hidden_states)
    
    # labels = 1-node_status[:,0]   
    # labels = torch.from_numpy(labels)
    # labels = labels.reshape(-1,1)
    # labels = labels.to(device).float()
    # loss = loss_fc(scores, labels)
    # optimizer.zero_grad()
    # loss.backward()
    # optimizer.step()
    # hidden_states = hidden_states.detach()
    # Pick top k that are not isolated 
    probability = k_node_sample_prob(scores)
    counter =k
    isolated_index = np.where(node_status[:,3] == 1)[0]

    # don't calculate loss on isolated nodes
    # scores_non_isolated= scores[np.where(node_status[:,3] == 0)[0]]
    # labels_non_isolated = labels[np.where(node_status[:,3] == 0)[0]]

    # don't intervene on isolated node
    probability[isolated_index]=0
    while counter>0:
        prev_test_positive = torch.where(node_features[:,5]==1)[0]
        node_features[prev_test_positive,5]=0
        node_features[prev_test_positive,7]=1
        index = np.argmax(np.random.multinomial(1,probability.cpu().detach().numpy().reshape(-1,),1))
        node_features[index,4]=0 
        if node_status[index,2]==1:
            node_status[index,2]=0
            node_status[index,-1]=1
            node_features[index,5]=1
            # detection_rate[t]+=1
        elif node_status[index,1]>0:
            node_status[index,1]=0
            node_status[index,-1]=1
            node_features[index,5]=1
            # detection_rate[t]+=1
        else:
            node_features[index,6]=1
        probability[index]=0
        counter-=1





-------------------
Epoch 0
-------------------
t =  0
0.988 percent healthy nodes
-------------------
t =  1
0.972 percent healthy nodes
-------------------
t =  2
0.95 percent healthy nodes
-------------------
t =  3
0.896 percent healthy nodes
-------------------
t =  4
0.804 percent healthy nodes
-------------------
t =  5
0.644 percent healthy nodes
-------------------
t =  6
0.469 percent healthy nodes
-------------------
t =  7
0.304 percent healthy nodes
-------------------
t =  8
0.181 percent healthy nodes
-------------------
t =  9
0.103 percent healthy nodes
-------------------
t =  10
0.055 percent healthy nodes
-------------------
t =  11
0.032 percent healthy nodes
-------------------
t =  12
0.026 percent healthy nodes
-------------------
t =  13
0.021 percent healthy nodes
-------------------
t =  14
0.019 percent healthy nodes
-------------------
Epoch 1
-------------------
t =  0
0.986 percent healthy nodes
-------------------
t =  1
0.971 percent healthy nodes
-----

/opt/conda/conda-bld/pytorch_1623448238472/work/aten/src/ATen/native/cuda/Loss.cu:111: operator(): block: [1,0,0], thread: [28,0,0] Assertion `input_val >= zero && input_val <= one` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [2]:
i

26

In [6]:
node_features[1,4]

IndexError: index 8 is out of bounds for dimension 1 with size 8

In [2]:
node_features.dtype

torch.float32

In [34]:
prob = prob.reshape(-1,)

In [46]:
list(prob)

[0.00024122889,
 0.00011093928,
 0.00025212072,
 1.174647e-05,
 3.1682615e-05,
 0.00041491742,
 0.00019148944,
 1.174647e-05,
 0.0002928084,
 0.00034966218,
 7.823803e-05,
 0.00010563956,
 0.00024239183,
 0.00022602471,
 0.000314465,
 9.793959e-05,
 0.0001065037,
 0.00019966462,
 0.0002024943,
 6.934314e-05,
 0.00024353423,
 0.00022097229,
 0.00016556896,
 4.1872332e-05,
 0.00027231546,
 0.00037292994,
 0.00026903264,
 0.0002682167,
 0.00026926692,
 0.00022171784,
 0.00011967594,
 0.00019899243,
 0.00025697506,
 1.923088e-05,
 0.0002176527,
 7.247203e-05,
 1.174647e-05,
 1.34724605e-05,
 1.174647e-05,
 0.00028443185,
 0.00057966856,
 0.00025330836,
 0.00012231446,
 0.00030814964,
 0.00019905032,
 1.174647e-05,
 5.3040043e-05,
 0.0003103231,
 0.00015062133,
 0.00030600725,
 0.000266664,
 0.00027080672,
 6.654754e-05,
 1.174647e-05,
 0.00011706647,
 0.0001500975,
 0.00012660785,
 0.00029539558,
 0.0005258089,
 9.662005e-05,
 0.00025037467,
 0.00023983639,
 2.2526738e-05,
 0.0001567717,
 

In [47]:
index = np.random.multinomial(1,prob,1)

In [51]:
np.argmax(index)

4532

In [1]:
import torch
from torch_geometric.loader import DataLoader,RandomNodeSampler, NeighborLoader
from torch_geometric.data import Data
from models import *
from utils import *
from preprocessing import *
import networkx as nx

In [2]:
loss = nn.BCELoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)

In [8]:
input

tensor([-0.6153,  0.3131,  0.9521], requires_grad=True)

In [10]:
m = nn.Sigmoid()
m(input)

tensor([0.3508, 0.5776, 0.7215], grad_fn=<SigmoidBackward>)

In [11]:
target

NameError: name 'target' is not defined